# Pizza Delivery


## Game

There are several students who wants pizza. There is one courier, asked to deliver the pizzas. For every student he is supposed to reach the student's favourite pizza restaurant, grab the pizza and deliver it to student. 

Students and pizza location map is given as a graph. Students and pizza shops are located in different nodes and there are edges, connecting nodes. Some edges are *loose* meaning that if you cross them once, you should wait fixed amount of time before crossing them again. Courier can only pickup a pizza if it is on the same node as pizza shop.

Courier must deliver some student's pizzas sooner than other ones (because of anger management).

On each unit of time, courier can do one of below ***actions***:
*   move to one of neigbour nodes from its own location
*   stay on its own location (maybe there is a loose edge and we are waiting for it)

We are supposed to calculate the minimum time that all pizza's are delivered.


## Implementation

### Node

This class represent one node of pizza map graph. There is an integer identifier for node called id.

In [1]:
class Node:
    def __init__(self, id) -> None:
        self.id = id
    
    def __eq__(self, another_node) -> bool:
        return self.id == another_node.id
    
    def __str__(self) -> str:
        return f"""Node:
            id: {self.id}
        """

    def __repr__(self) -> str:
        return self.__str__()


### Edge

This class represent an edge connecting two nodes. Edges can be loose and if you traverse them, they are unavailable for period of length ***loose_time***

In [2]:
class Edge:
    def __init__(self, id: int, left_node: Node, right_node: Node, is_loose: bool, loose_time: int) -> None:
        self.id = id
        self.left_node = left_node
        self.right_node = right_node
        self._is_loose = is_loose
        self.loose_time = loose_time
    
    def set_loose_time(self, loose_time: int):
        self._is_loose = True
        self.loose_time = loose_time
    
    def get_loose_time(self):
        return self.loose_time
    
    def get_other_node(self, node: Node):
        if self.left_node == node:
            return self.right_node
        return self.left_node

    def is_loose(self) -> bool:
        return self._is_loose

    def __eq__(self, another_edge) -> bool:
        return self.id == another_edge.id
    
    def __str__(self) -> str:
        return f"""Edge:
            id: {self.id},
            left node id: {self.left_node.id},
            right node id: {self.right_node.id},
        """

    def __repr__(self) -> str:
        return self.__str__()


### Student

Every student is location in a specific node and wants a pizza from a pizza shop in another specific node.

In [3]:
class Student:
    def __init__(self, id: int, location: Node, favourite_pizza_shop_node: Node) -> None:
        self.id = id
        self.favourite_pizza_shop_node = favourite_pizza_shop_node
        self.location = location
    
    def __eq__(self, another_student) -> bool:
        return self.id == another_student.id

    def __str__(self) -> str:
        return f"""Student:
            id: {self.id},
            location: {self.location.id},
            pizza shop location: {self.favourite_pizza_shop_node.id}
        """

    def __repr__(self) -> str:
        return self.__str__()

### PizzaMap

This class provides basic functionalites to store and get different map aspects. Functions must be fast so there are some data redundancies.

In [4]:
from typing import Dict, List


class PizzaMap:
    def __init__(self) -> None:
        self.nodes = {}
        self.edges = {}
        self.connections = {}
        self.students = {}
        self.students_location = {}
        self.students_favourite_pizza_locations = {}
        self.preceding_constraint = {}
        self.init_courier_location = None
        self.pizza_shop_nodes = set()
        

        self.students: Dict[int, Student]
        self.init_courier_location: Node
        self.nodes: Dict[int, Node]
        self.edges: Dict[int, Edge]
        self.connections: Dict[int, List[Edge]]
        self.preceding_constraint: Dict[int, List[int]]

    def add_node(self, node: Node):
        self.nodes[node.id] = node
    
    def add_edge_to_node(self, node: Node, edge_id: int):
        node_edges = self.connections.get(node.id, [])
        node_edges.append(edge_id)
        self.connections[node.id] = node_edges

    def add_edge(self, edge: Edge):
        self.edges[edge.id] = edge
        self.add_edge_to_node(edge.left_node, edge.id)
        self.add_edge_to_node(edge.right_node, edge.id)

    def add_student(self, student: Student):
        self.students[student.id] = student
        self.students_location[student.location.id] = student
        self.students_favourite_pizza_locations[student.favourite_pizza_shop_node.id] = student
        self.pizza_shop_nodes.add(student.favourite_pizza_shop_node.id)
    
    def add_preceding_constraint(self, before_student: Student, after_student: Student):
        self.preceding_constraint[after_student.id] = self.preceding_constraint.get(
            after_student.id,
            []
        ) + [before_student.id]

    def set_edge_loose(self, edge_id: int, loose_time: int):
        self.get_edge(
            edge_id=edge_id
        ).set_loose_time(
            loose_time=loose_time
        )
    
    def set_init_courier_location(self, courier_location: Node):
        self.init_courier_location = courier_location
    
    def get_edge(self, edge_id: int) -> Edge:
        return self.edges[edge_id]
    
    def get_node(self, node_id: int) -> Node:
        return self.nodes[node_id]

    def get_student(self, student_id: int) -> Student:
        return self.students[student_id]

    def get_student_by_location(self, node: Node) -> Student:
        return self.students_location[node.id]
    
    def get_student_by_favourite_pizza_location(self, node: Node) -> Student:
        return self.students_favourite_pizza_locations[node.id]
    
    def get_preceding_students_id(self, student: Student) -> List[Student]:
        return self.preceding_constraint.get(student.id, [])

    def get_neighbour_egdes(self, node: Node):
        return [self.edges[edge_id] for edge_id in self.connections[node.id]]

    def get_init_courier_location(self):
        return self.init_courier_location
    
    def has_pizza_shop(self, node: Node) -> bool:
        return node.id in self.pizza_shop_nodes

    def has_student(self, node: Node) -> bool:
        return node.id in self.students_location


## Input

The graph, students, initial courier location, loose edges and student priorities is given in a text input. We wrote a class to parse the input.

*   **n** and **m** are given in first line denoting number of nodes and edges.
*   then there are **m** lines with two integer **i** and **j**, denoting that there is an edge between i'th and j'th node.
*   next line is **h** denoting number of loose edges.
*   next **h** lines are a pair of integers **k**, **p** each. showing that k'th edge is loose and if you cross it, it will be unavailable for **p** units of time
*   next line is **v** denoting the courier initial node
*   next, there is **s**, the total number of students
*   the **t**'th of next **s** lines is a pair **e** and **c** showing that student **t** is located in **e**'th node and wants pizza from a shop in **c**'th node
*   then there is **x**. number of precedence pairs
*   at the end, there are **x** lines **z** and **y**, each denoting that **z**'th student's pizza must arrive before **y**'th student's

example:

```
    5 4
    1 2
    2 3
    3 4
    3 5
    1
    3 2
    1
    2
    4 3
    5 2
    1
    1 2
```

![example 1](./media/AI-CA1-Spring%201402-3.jpg) 

In [5]:
class PizzaMapSerializer:
    def __init__(self, file_path: str) -> None:
        self.map = PizzaMap()
        self.file = open(file_path, 'r')
    
    def parse_node_and_edge_count(self):
        self.n, self.m = map(int, self.file.readline().split())

        for i in range(self.n):
            self.map.add_node(Node(i))

    def parse_edges(self):
        for i in range(self.m):
            left_node_id, right_node_id = map(lambda x: int(x) - 1, self.file.readline().split())
            left_node, right_node = self.map.get_node(left_node_id), self.map.get_node(right_node_id)
            self.map.add_edge(Edge(
                id=i,
                left_node=left_node,
                right_node=right_node,
                is_loose=False,
                loose_time=None
            ))
    
    def parse_loose_edges(self):
        loose_edge_count = int(self.file.readline())

        for i in range(loose_edge_count):
            edge_id, loose_time = map(int, self.file.readline().split())
            edge_id -= 1
            self.map.set_edge_loose(edge_id=edge_id, loose_time=loose_time)
    
    def parse_courier_location(self):
        courier_node_id = int(self.file.readline()) - 1
        self.map.set_init_courier_location(self.map.get_node(courier_node_id))

    def parse_students(self):
        students_count = int(self.file.readline())

        for i in range(students_count):
            location_node_id, favouire_pizza_shop_node_id = map(
                lambda x: int(x) - 1,
                self.file.readline().split()
            )
            self.map.add_student(
                Student(
                    i,
                    self.map.get_node(location_node_id),
                    self.map.get_node(favouire_pizza_shop_node_id)
                )
            )

    def parse_delivery_order(self):
        preceding_constraints_count = int(self.file.readline())

        for i in range(preceding_constraints_count):
            before_student_id, after_student_id = map(
                lambda x: int(x) - 1,
                self.file.readline().split()
            )
            self.map.add_preceding_constraint(
                self.map.get_student(before_student_id),
                self.map.get_student(after_student_id)
            )

    def parse(self) -> PizzaMap:
        self.parse_node_and_edge_count()
        self.parse_edges()
        self.parse_loose_edges()
        self.parse_courier_location()
        self.parse_students()
        self.parse_delivery_order()
        return self.map

## Solution

We want to solve this problem and code. The solution must be as fast as possible. There are some tests and each test has its own time limit. 

### Algorithm

We build a new graph from pizza map graph and we call it ***State Graph***. We will find the minimum path from ***initial state*** to a ***good state*** in constructed state graph using different best path finding algorithms like BFS, IDS, A*. 
 

### State

Each node in state graph denotes a game state including ***courier location***, ***students waiting for pizza***, ***unavailable edges (plus time left to get available)*** and ***courier's pizza on hand (maybe none)*** along with pre give pizza map. State ***u*** is connected to state ***v*** by a directed edge if there is a action in ***x*** for courier that will change the state to ***y***.

***Initial state*** of game is like below:
*   courier is on its initial location (given in input. we will retreive it from pizza map class)
*   all students are waiting pizza and no pizza has been delivered
*   there is no unavailable edge as courier didnt cross any edges yet
*   courier doesnt have any pizza in hand

***Good state (end state)*** of game is a state with no students waiting for pizza (all pizzas are delivered).

In [6]:
from typing import Set
from queue import Queue


class DeliveryState:
    ID = 0

    class NoPathFound(Exception):
        pass

    def __init__(
        self,
        id: int,
        parent,
        pizza_map: PizzaMap,
        courier_location: Node,
        unsatisfied_student_ids: Set[int],
        loose_edges: Dict[int, int],
        courier_holding_pizza_node_id: int
    ) -> None:
        self.id = id
        self.parent = parent
        self.map = pizza_map
        self.courier_location = courier_location
        self.unsatisfied_student_ids = unsatisfied_student_ids
        self.loose_edges = loose_edges
        self.courier_holding_pizza_node_id = courier_holding_pizza_node_id
        self.traveled = []
        self.traveled: List[Node]

    def get_preceding_students(self, student: Student):
        initial_preceding_students_ids = self.map.get_preceding_students_id(student)
        
        res = []
        for unsatisfied_student_id in self.unsatisfied_student_ids:
            if unsatisfied_student_id in initial_preceding_students_ids:
                res.append(unsatisfied_student_id)

        return res

    def find_min_courier_path_to_node(self, destination: Node):
        queue = Queue()
        queue.put(self.courier_location)
        
        dist = {}
        dist[self.courier_location.id] = (0, None)

        while not queue.empty():
            node = queue.get()
            dist_to_node, _ = dist[node.id]
            
            for edge in self.map.get_neighbour_egdes(node):
                if self.loose_edges.get(edge.id, 0) > dist_to_node:
                    continue

                neighbour = edge.get_other_node(node)
                
                if dist.get(neighbour.id, None) is not None:
                    continue
                
                dist[neighbour.id] = (dist_to_node + 1, edge)

                if neighbour == destination:
                    res = []
                    traversed_node = neighbour
                    traversed_edge = edge
                    while traversed_node != self.courier_location:
                        res.append(traversed_edge)
                        traversed_node = traversed_edge.get_other_node(traversed_node)
                        traversed_edge = dist[traversed_node.id][1]
                    return list(reversed(res))

                queue.put(neighbour)

    def is_good(self):
        return len(self.unsatisfied_student_ids) == 0

    def tick(self):
        pop_ids = []
        for loose_edge_id in self.loose_edges:
            time_remaining = self.loose_edges[loose_edge_id]
            if time_remaining == 1:
                pop_ids.append(loose_edge_id)
            else:
                self.loose_edges[loose_edge_id] -= 1

        for pop_id in pop_ids:
            self.loose_edges.pop(pop_id)

    def update_unstatisfied_students(self):
        if not self.map.has_student(
            self.courier_location
        ):
            return
        
        if self.courier_holding_pizza_node_id is None:
            return
        
        student = self.map.get_student_by_location(self.courier_location)

        if student.favourite_pizza_shop_node.id != self.courier_holding_pizza_node_id:
            return
        
        self.courier_holding_pizza_node_id = None
        self.unsatisfied_student_ids.remove(student.id)

    def update_courier_pizza(self):
        if not self.map.has_pizza_shop(
            self.courier_location
        ):
            return
        
        if self.courier_holding_pizza_node_id is not None:
            return
        
        student = self.map.get_student_by_favourite_pizza_location(
            self.courier_location
        )

        if not student.id in self.unsatisfied_student_ids:
            return

        if len(self.get_preceding_students(student)) > 0:
            return
        
        self.courier_holding_pizza_node_id = self.courier_location.id

    def loose_edge_traveled(self, loose_edge: Edge):
        self.loose_edges[loose_edge.id] = loose_edge.get_loose_time()

    def travel_edge(self, edge: Edge):
        new_node = edge.get_other_node(self.courier_location)
        self.courier_location = new_node

        self.update_unstatisfied_students()
        self.update_courier_pizza()
        self.tick()
        if edge.is_loose():
            self.loose_edge_traveled(edge)

        self.traveled.append(new_node)

    def stay(self):
        stay_time = 0
        while len(self.loose_edges) > 0:
            self.tick()
            stay_time += 1
            self.traveled.append(self.courier_location)

        return stay_time
    
    def travel_to_node(self, destination_node):
        path_to_dest = self.find_min_courier_path_to_node(destination_node)
        
        if path_to_dest is None:
            raise DeliveryState.NoPathFound()
        
        for edge in path_to_dest:
            self.travel_edge(edge)

        return path_to_dest

    def get_neighbours(self, ):
        neighbours = []
        neighbours: List[DeliveryState]
        
        possible_desitnations = []
        if self.courier_holding_pizza_node_id is not None:
            student = self.map.get_student_by_favourite_pizza_location(
                self.map.get_node(
                    self.courier_holding_pizza_node_id
                )
            )
            possible_desitnations.append(student.location)

        else:
            for student_id in self.unsatisfied_student_ids:
                student = self.map.get_student(
                    student_id
                )
                if len(self.get_preceding_students(student)) > 0:
                    continue

                possible_desitnations.append(student.favourite_pizza_shop_node)

        for destination_node in possible_desitnations:
            new_state = DeliveryState.deepcopy(self)

            try:
                travel_path = new_state.travel_to_node(destination_node)
            except DeliveryState.NoPathFound:
                pass
            else:
                neighbours.append((new_state, len(travel_path)))

        # if there is a loose edge, we can stay and wait for it
        if len(self.loose_edges) > 0:
            stay_state = DeliveryState.deepcopy(self)
            stay_time = stay_state.stay()
            neighbours.append((stay_state, stay_time))

        return neighbours

    @staticmethod
    def get_new_id():
        DeliveryState.ID += 1
        return DeliveryState.ID

    @staticmethod
    def deepcopy(state):
        state: DeliveryState
        new_state = DeliveryState(
            DeliveryState.get_new_id(),
            state,
            state.map,
            state.courier_location,
            state.unsatisfied_student_ids.copy(),
            state.loose_edges.copy(),
            state.courier_holding_pizza_node_id
        )
        new_state: DeliveryState
        return new_state

    def identifier(self):
        return (
            self.courier_location.id,
            tuple(self.loose_edges.items()),
            tuple(self.unsatisfied_student_ids),
            self.courier_holding_pizza_node_id
        )
    
    def __hash__(self) -> int:
        return hash(self.identifier())

    def __str__(self) -> str:
        return f"""State {self.id} {hash(self)} {self.identifier()}:
            courier_location: {self.courier_location.id},
            remaining_students: {self.unsatisfied_student_ids},
            courier_pizza: {self.courier_holding_pizza_node_id},
            loose_edges: {self.loose_edges},
            is good: {self.is_good()},
            parent: {self.parent.id if self.parent is not None else None}
            traveled: {[node.id for node in self.traveled]}
        """
    
    def __repr__(self) -> str:
        return self.__str__()
    
    def __lt__(self, another_state):
        another_state: DeliveryState
        return self.id < another_state.id

### Performance issues

There will be too many state if we want to build all possible states. So we have to remove and prevent some ***irrelevant*** states. For example if courier holds a pizza belonging to student s and it is one node away from s then going to another node is irrelevant and irrational and this move surely wont be one the best scenario. 

To find relevant next states from a random state ***S*** (get_neighbours method), we will check possible actions for courier:
*   If courier is holding pizza for student ***s*** then it travel must travel to ***s*** location. So we will find the best route to ***s*** (with bfs) and we will travel all the way to s from current state. With crossing each edge, we will update the whole state until we get to ***s***. The final state ***F*** will be a neighbour of ***S*** with an edge weighted equal to courier path length. 
    *    ***Note*** that we wont wait for any loose edge, this will be handled in another transition. Loose edges are only considered in our route finding bfs fron courier location to ***s***.
  
*   If courier is not holding a pizza, we can choose any student having no preceding unstatisfied student and we will have to travel to his/her favourie pizza location to grab the pizza. So again, we will have a destination node like previous part. 
*   Courier can wait in its location. With a closer look we can find out that when courier is waiting, there is only one reason. A loose edge. So if we are waiting, we should wait until atleast one loose edge is available again.
    *   ***Note*** that when courier is traveling to node ***s***, and it waits ***n*** units of time in different parts of path, it could wait ***n*** units in its starting location and it can travel again to ***s*** after that


With above considerations, our algorithm will be much much faster with smaller state graph

### MinPathFinder

This is an interface for all min path finding algroithms we will write 

In [7]:
class MinPathFinder:
    def __init__(self, map: PizzaMap, init_state: DeliveryState) -> None:
        self.map = map
        self.init_state = init_state

    def check_end(self, state: DeliveryState): 
        if not state.is_good():
            return None

        result = []
        tmp_state = state
        while tmp_state is not None:
            for traveled_node in reversed(tmp_state.traveled):
                result.append(traveled_node.id + 1)
            tmp_state = tmp_state.parent

        result.append(self.init_state.courier_location.id + 1)
        return list(reversed(result))

    def find(self) -> List[int]:
        pass

### BFS

This is actually not a bfs (cause state graph is weighted), but there are not much different.

In [8]:
from queue import PriorityQueue
import datetime


class BFS(MinPathFinder):
    def find(self):
        queue = PriorityQueue()
        queue.put((0, self.init_state))
        visited = {}

        while not queue.empty():
            dist, child = queue.get()
            child: DeliveryState
            visited[hash(child)] = True
            
            check_end_result = self.check_end(child)
            if check_end_result is not None:
                return check_end_result
            
            for neighbour, cost in child.get_neighbours():
                if visited.get(hash(neighbour), None) is not None:
                    continue

                queue.put((dist + cost, neighbour))

        return []


### IDS

IDS implementation of min path finder.

In [9]:
class IDS(MinPathFinder):
    def dls(self, state: DeliveryState, limit: int):
        if limit < 0:
            return None

        check_end_result = self.check_end(state)
        if check_end_result is not None:
            return check_end_result

        for neighbour, cost in state.get_neighbours():
            neighbour_check_res = self.dls(neighbour, limit - cost)

            if neighbour_check_res is not None:
                return neighbour_check_res

        return None

    def find(self) -> List[int]:
        depth = 0
        result = self.dls(self.init_state, depth)
        while result is None:
            depth += 1
            result = self.dls(self.init_state, depth)
        
        return result

### A*

We will use A* and weighted A* search to implement another min path finder. 

Our heuristic function is simple. Number of ***unsatisfied students***. ***Note*** that our hueristic is surely consistent because for every new satisfied student there must be at least one action (edge) (cause two students cant be located in same node).

***Note*** that we can use alpha = 2 and alpha = 1.5 for our weighted A* because our hueristic will remain consitent.

In [10]:
from queue import PriorityQueue


def astar_setup(alpha=1):
    class Astar(MinPathFinder):
        def hueristic(self, state: DeliveryState):
            return alpha * len(state.unsatisfied_student_ids)

        def find(self):
            queue = PriorityQueue()
            queue.put((0 + self.hueristic(self.init_state), self.init_state))

            distances = {}
            distances[hash(self.init_state)] = 0

            while not queue.empty():
                _, child = queue.get()
                child: DeliveryState
                distance = distances[hash(child)]

                check_end_result = self.check_end(child)
                if check_end_result is not None:
                    return check_end_result

                for neighbour, cost in child.get_neighbours():
                    current_distance = distances.get(hash(neighbour), None)
                    if current_distance is not None and current_distance <= distance + cost:
                        continue

                    queue.put((distance + cost + self.hueristic(neighbour), neighbour))
                    distances[hash(neighbour)] = distance + cost

            return []
    
    return Astar


## Test

We have wrote a tester class to test finders and delivery states

In [11]:
import os
from typing import Type, Tuple


class Tester:

    class TestFailed(Exception):
        pass


    def __init__(self, tests_path: str, finder: Type[MinPathFinder]):
        self.tests_path = tests_path
        self.finder = finder

    def run_test(self, input_file: str) -> Tuple[List[int], float]:
        pizza_map = PizzaMapSerializer(input_file).parse()

        init_state = DeliveryState(
            DeliveryState.ID,
            None,
            pizza_map,
            pizza_map.get_init_courier_location(),
            set(pizza_map.students.keys()),
            {},
            None
        )

        start = datetime.datetime.now()
        result = self.finder(pizza_map, init_state).find()
        end = datetime.datetime.now()

        return (result, (end - start).total_seconds())

    def test(self, input_path: str, output_path: str):
        result, execution_time = self.run_test(input_path)

        with open(output_path, 'r') as output_file:
            correct_result = list(map(int, output_file.readline().split(' ')))

            if len(correct_result) == len(result):
                return (True, result, execution_time)
            else:
                return (False, result, execution_time)

    def tests(self) -> List[Tuple[bool, Tuple[int, float]]]:
        result = {}

        test_directories = []
        for test_directory in os.scandir(self.tests_path):
            if not test_directory.is_dir():
                return
            
            test_directories.append((test_directory.name, test_directory))
        
        test_directories.sort()
        test_directories = test_directories
        for name, test_directory in test_directories:
            result[name] = self.test(test_directory.path + '/in.txt', test_directory.path + '/out.txt')

        return result


### Bench Mark

In [12]:
class BCOLORS:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'


finders = {
    "BFS": BFS,
    "IDS": IDS,
    "A*": astar_setup(),
    "1.5 - A*": astar_setup(1.5),
    "2 - A*": astar_setup(2)
}

finders_result = {}

for finder_name in finders:
    tests_result = Tester(
        "./tests/",
        finders[finder_name]
    ).tests()
    print(finder_name + ":")
    for test in tests_result:
        if tests_result[test][0] is True:
            print(
                "\t" + f"{test}" + " " + BCOLORS.OKGREEN + f"Accepted" + BCOLORS.ENDC + ". " + f"Executed in {tests_result[test][2]}."
            )
        else:
            print("\t" + f"{test}" + " " + BCOLORS.FAIL + f"Failed" + BCOLORS.ENDC + ".")

    finders_result[finder_name] = tests_result



BFS:
	Test0 Accepted. Executed in 0.000355.
	Test1 Accepted. Executed in 0.003192.
	Test2 Accepted. Executed in 0.015576.
	Test3 Accepted. Executed in 2.832304.
	Test4 Accepted. Executed in 0.002529.
	Test5 Accepted. Executed in 0.009333.
	Test6 Accepted. Executed in 0.009873.
IDS:
	Test0 Accepted. Executed in 0.000984.
	Test1 Accepted. Executed in 0.012191.
	Test2 Accepted. Executed in 0.1796.
	Test3 Accepted. Executed in 202.194153.
	Test4 Accepted. Executed in 0.014513.
	Test5 Accepted. Executed in 0.069004.
	Test6 Accepted. Executed in 0.06332.
A*:
	Test0 Accepted. Executed in 0.000225.
	Test1 Accepted. Executed in 0.001458.
	Test2 Accepted. Executed in 0.003969.
	Test3 Accepted. Executed in 0.095274.
	Test4 Accepted. Executed in 0.001773.
	Test5 Accepted. Executed in 0.005899.
	Test6 Accepted. Executed in 0.002882.
1.5 - A*:
	Test0 Accepted. Executed in 0.000218.
	Test1 Accepted. Executed in 0.001345.
	Test2 Accepted. Executed in 0.003744.
	Test3 Accepted. Executed in 0.099045.
	T

### Bench Mark Result

|         | Test 1   | Test 2   | Test 3   |
|---------|----------|----------|----------|
|   BFS   | 0.002752 | 0.018602 | 2.331865 |
|   IDS   | 0.013393 | 0.175650 | 220.2450 |
|    A*   | 0.001868 | 0.005586 | 0.155430 |
|  1.5A*  | 0.001319 | 0.003650 | 0.115968 |
|   2A*   | 0.001336 | 0.003388 | 0.111748 |

### Final Note

We have tested different search algorithms (informed and uninformed) on our infinite graph. As you can see A* has performed much better.